In [ ]:
import os
import cv2
import pickle
import numpy as np
import pandas as pd

In [ ]:
def breaker(num: int = 50, char: str = "*") -> None:
    print("\n" + num*char + "\n")


def preprocess(image: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)


def get_statistics(path: str, names: np.ndarray, sizes: list, broken_images: list) -> list:
    r_means, g_means, b_means, r_stds, g_stds, b_stds = [], [], [], [], [], []

    i = 0
    for size in sizes:
        r_mean, g_mean, b_mean, r_std, g_std, b_std = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for name in names:
            if name in broken_images:
                pass
            else:
                main_image = preprocess(cv2.imread(os.path.join(path, name), cv2.IMREAD_COLOR))
                image = cv2.resize(src=main_image, dsize=(size, size), interpolation=cv2.INTER_AREA)
                r_mean += image[:, :, 0].mean()
                g_mean += image[:, :, 1].mean()
                b_mean += image[:, :, 2].mean()
                r_std  += image[:, :, 0].std()
                g_std  += image[:, :, 1].std()
                b_std  += image[:, :, 2].std()
        r_means.append(r_mean / len(names))
        g_means.append(g_mean / len(names))
        b_means.append(b_mean / len(names))

        r_stds.append(r_std / len(names))
        g_stds.append(g_std / len(names))
        b_stds.append(b_std / len(names))
    
    return r_means, g_means, b_means, r_stds, g_stds, b_stds 

In [ ]:
train_df = pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")

broken_images = [filename for filename in train_df.image if filename not in os.listdir("../input/sorghum-id-fgvc-9/train_images")]
for broken_image in broken_images:
    index = train_df.index[train_df.image == broken_image]
    train_df = train_df.drop(index=index)
    
filenames = train_df.iloc[:, 0].copy().values

In [ ]:
sizes = [256, 384, 512, 768, 1024]

r_means, g_means, b_means, r_stds, g_stds, b_stds = get_statistics("../input/sorghum-id-fgvc-9/train_images", filenames, sizes, broken_images)

In [ ]:
breaker()
for i in range(len(sizes)):
    print(f"Red Channel Mean ({sizes[i]})   : {r_means[i]:.5f}")
    print(f"Green Channel Mean ({sizes[i]}) : {g_means[i]:.5f}")
    print(f"Blue Channel Mean ({sizes[i]})  : {b_means[i]:.5f}")
    print("\n")
    print(f"Red Channel Std ({sizes[i]})    : {r_stds[i]:.5f}")
    print(f"Green Channel Std ({sizes[i]})  : {g_stds[i]:.5f}")
    print(f"Blue Channel Std ({sizes[i]})   : {b_stds[i]:.5f}")
    breaker()